In [1]:
#load packages
import os.path
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

import gensim 
from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet

import gensim.corpora as corpora
from gensim.corpora import Dictionary

from gensim import matutils, models

import pyLDAvis.gensim
import string
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

import spacy
# Load English model for SpaCy
nlp = spacy.load("en_core_web_sm")

C:\Users\NIY\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\NIY\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
C:\Users\NIY\Anaconda3\lib\site-packages\nltk\lm\vocabulary.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Counter, Iterable


In [2]:
def get_most_prob_topic(unseen_document, model):
    """
    Given an unseen_document, and a trained LDA model, this function
    finds the most likely topic (topic with the highest probability) from the 
    topic distribution of the unseen document and returns the best topic with 
    its probability. . 
    
    Parameters
    ------------
    unseen_document : (str) 
        the document to be labeled with a topic
    model : (gensim ldamodel) 
        the trained LDA model
    
    Returns: 
    -------------
        (str) a string of the form 
        `most likely topic label:probability of that label` 
    
    Examples:
    ----------
    >> get_most_prob_topic("The research uses an HMM for discovering gene sequence.", 
                            model = lda)
    Science and Technology:0.435
    """    
    #preprocess 
    preprocessed = preprocess(unseen_document)
    
    #process into a doc_term_matrix
    corpus = preprocessed
    bow_vector = dictionary.doc2bow(corpus)
    
    
    #find simliar articals
    simliar_artical = model[bow_vector]
    
    
    #change the topic name and sort tuple
    items = []

    for idx, prob in simliar_artical:
        items.append((topic_labels[idx], prob))

    items = sorted(items, key=lambda x: x[1], reverse=True)
    out = items[0][0]
    return out

In [3]:
def preprocess(txt, 
               min_token_len = 3, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']): 
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Parameters
    -------------
    text : (str) 
        the text to be preprocessed
    min_token_len : (int) 
        min_token_length required
    irrelevant_pos : (list) 
        a list of irrelevant pos tags
    
    Returns
    -------------
    (str) the preprocessed text
    """
    #change to string
    txt = str(txt)

    #weird character
    txt = re.sub('[^a-zA-Z0-9]+',' ', txt)

    #multiple spaces
    txt = re.sub(' +',' ', txt)
    
    #to lower
    txt = txt.lower()
    
    #remove numbers
    txt = re.sub('\d+', '', txt)
    

    #spacy remove stopwords
    text_tokens = nlp(txt)
    tokens = [word for word in text_tokens if not word in stopwords]
    
    #lenght
    tokens = [word for word in tokens if len(word)>min_token_len]
    
    #remove irrelevant_pos
    tokens = [word for word in tokens if not word.pos_ in irrelevant_pos]
    
    #lemmentatiation
    tokens = [word.lemma_ for word in tokens]
    
    return tokens
    

<>:34: DeprecationWarning: invalid escape sequence \d
<>:34: DeprecationWarning: invalid escape sequence \d
<>:34: DeprecationWarning: invalid escape sequence \d
<ipython-input-3-f9315f9f9162>:34: DeprecationWarning: invalid escape sequence \d
  txt = re.sub('\d+', '', txt)


In [4]:
def remove_stopwords(tokens):
    tokens = [word for word in tokens if not word in stopwords]
    return tokens

In [5]:
def join_text(tokens):
    return " ".join(tokens)

In [6]:
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
# bigram_mod = gensim.models.phrases.Phraser(bigram)

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

In [7]:
topic_labels = {0:'01',
                1:'02',
                2:'03',
                3:'04',
                4:'05',
                5:'06',
                6:'07',
                7:'08',
                8:'09',
                9:'10',
                10:'11',
                11:'12',
                12:'13',
                13:'14',
                14:'15'}

## Load and merge data frames

In [26]:
global_path= "../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/make_processed/"

df_train_long = pd.read_csv(global_path+"train_dataset.csv")
df_resume_long = pd.read_csv(global_path+"english_clean_resumes.csv")


In [27]:
df_train_long.head(2)

,Unnamed: 0,employee_code,original_hire_date,max_hire_date,gender,job_title,position_status,worker_category,birth_year,report_date_week_ending_,...,language,resume_found,months_high_perf,months_with_perf,hp_perc,hp_class,perf_found,rehired_,referral_flag,exclusion_code
0,2825,MZE,"Saturday, May 18, 2019",2019-05-18,Female,SALES ASSOCIATE,Active,Permanent Full Time,1997,2019-05-24,...,English,1.0,0.0,3.0,0.0,0.0,1.0,NaN,1.0,06-Pass
1,2639,MRU,"Friday, February 22, 2019",2019-02-22,Male,SALES ASSOCIATE,Active,Permanent Full Time,1991,2019-02-22,...,English,1.0,0.0,3.0,0.0,0.0,1.0,NaN,NaN,06-Pass


In [28]:
df_resume_long.head(2)

,Unnamed: 0,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text
0,0,"Abbasi, Samiee Z",N42,WW 374,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Samiee Zahid Abbasi 61 Muscat Crescent Ajax, ON L1Z 0B8 Phone: 647-285-3809 E-mail: sabbasii@ic...","['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',...",English,docx,Zahid Muscat Crescent look build great work fastpaced environment skill Abilities read write spe...
1,1,"Abdi, Melika",N3O,TB 146,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkills\n\nExperien...,"Skills Experience MELIKA ABDI Carmelo Avenue Coquitlam, BC V3B7M9 Ph: 6047049487 melikaabdi94@g...","['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',...",English,pdf,Skills Experience Carmelo Avenue Ph ability work pressure effective leadership managemet Teamwor...


In [29]:
df_resume_long.shape

(496, 10)

In [30]:
df_train = df_train_long[['employee_code', 'hp_class']]
df_resume = df_resume_long[['employee_code', 'clean_text']]

In [31]:
df_topic_model = pd.merge(df_train, df_resume)

## Topic modeling

### Full training set

In [32]:
sp = spacy.load('en_core_web_sm')
sp.Defaults.stop_words |= {'customer', 'skill', 'experience', 'complete', 'june', 'program', 'sale', 'july', 'october', 'september', 'high', 'able', 'meet', 'year', 'need', 'joan', 'mean', 'management', 'client', 'service', 'team', 'product', 'store', 'ensure', 'manager', 'manager', 'communication', 'company', 'environment', 'work', 'provide', 'knowledge', 'business', 'target', 'associate', 'good', 'maintain', 'april', 'nee', 'employee', 'include', 'november'}
stopwords = sp.Defaults.stop_words

In [33]:
df_topic_model['token'] = df_topic_model['clean_text'].apply(preprocess)

In [34]:
df_topic_model['token_2'] = df_topic_model['token'].apply(remove_stopwords)

In [35]:
corpus = df_topic_model["token"]
dictionary = corpora.Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [36]:
lda = models.LdaModel(corpus=doc_term_matrix, 
                      id2word=dictionary, 
                      num_topics=15,
                      passes=20,
                      random_state=42)

In [37]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, doc_term_matrix, dictionary, sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.118758  0.013104       1        1  37.781784
1     -0.111079 -0.151423       2        1   1.189196
2      0.015460 -0.045971       3        1   2.064123
3     -0.201570  0.042706       4        1   0.538145
4     -0.010952  0.109798       5        1   2.370917
5     -0.157358  0.035714       6        1   0.864340
6     -0.014255  0.014018       7        1   1.803292
7      0.113588 -0.018572       8        1   9.683626
8      0.096336  0.005624       9        1   9.187202
9     -0.180555  0.011295      10        1   0.353501
10     0.023812 -0.032442      11        1   3.508947
11     0.096962  0.005806      12        1  14.094165
12     0.061979  0.013684      13        1   5.283538
13     0.088360  0.040792      14        1   6.642498
14     0.060514 -0.044133      15        1   4.634731, topic_info=          Term         Freq        Total Category  logprob  loglift
118       sale  1439.000000  1439.000000  Default  30.0000  30.0000
33    customer  1797.000000  1797.000000  Default  29.0000  29.0000
124      skill   763.000000   763.000000  Default  28.0000  28.0000
143       work   803.000000   803.000000  Default  27.0000  27.0000
216       food   140.000000   140.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
102    product    11.738145   468.116272  Topic15  -5.5876  -0.6143
377     client    10.650746   342.886383  Topic15  -5.6848  -0.4002
152  assistant     9.681101   127.620438  Topic15  -5.7802   0.4927
710    achieve     9.547875   137.505737  Topic15  -5.7941   0.4042
106    provide     9.476755   435.383301  Topic15  -5.8016  -0.7558

[1200 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4158      8  0.753727   aartex
0         1  0.434025  ability
0         2  0.004474  ability
0         3  0.017898  ability
0         5  0.017898  ability
...     ...       ...      ...
2034      5  0.252944    young
2034      6  0.084315    young
2034     11  0.084315    young
2034     12  0.084315    young
2034     13  0.252944    young

[3425 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

## Cluster Analysis

In [38]:
df_topic_model['token_join'] = df_topic_model['token_2'].apply(join_text)
df_topic_model['cluster'] = df_topic_model['token_join'].apply(get_most_prob_topic, model=lda)
df_topic_model_group = df_topic_model.drop(['employee_code', 'clean_text', 'token', 'token_2'], axis=1)
df_topic_model_group_hp = df_topic_model_group.groupby('hp_class')
df_topic_model_group_cluster = df_topic_model_group.groupby('cluster')
df_topic_model_group_all = df_topic_model_group.groupby(['hp_class', 'cluster'])
df_topic_model_group['cluster'] = df_topic_model_group["cluster"].astype('category')
df_topic_model_group['hp_class'] = df_topic_model_group["hp_class"].astype('category')

In [39]:
alt.Chart(df_topic_model_group).mark_bar().encode(
    x=alt.X('cluster:O'),
    y=alt.Y('count()'),
    color=alt.Color('hp_class:N')
)

alt.Chart(...)

In [40]:
alt.Chart(df_topic_model_group).mark_bar().encode(
    x=alt.X('cluster:O'),
    y=alt.Y('count()'),
    color=alt.Color('hp_class:N'),
    column=alt.Column('hp_class:N')
)

alt.Chart(...)

In [41]:
(df_topic_model_group_cluster['hp_class'].count()/df_topic_model_group_cluster.count()['hp_class'].sum())

cluster
01    0.364583
02    0.006944
03    0.024306
04    0.003472
05    0.031250
06    0.010417
07    0.020833
08    0.083333
09    0.086806
10    0.006944
11    0.034722
12    0.166667
13    0.052083
14    0.069444
15    0.038194
Name: hp_class, dtype: float64

In [42]:
df_topic_model_group_all.size()

hp_class  cluster
0.0       01         70
          02          1
          03          7
          04          1
          05          8
          06          3
          07          5
          08         17
          09         22
          10          2
          11          8
          12         36
          13         13
          14         17
          15          8
1.0       01         35
          02          1
          05          1
          07          1
          08          7
          09          3
          11          2
          12         12
          13          2
          14          3
          15          3
dtype: int64

### Only high performers 

In [44]:
df_topic_model_high = df_topic_model.query("hp_class == 1.0")

In [45]:
corpus = df_topic_model_high["token_2"]
dictionary = corpora.Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [46]:
lda_high = models.LdaModel(corpus=doc_term_matrix, 
                           id2word=dictionary, 
                           num_topics=15,
                           passes=30)

In [47]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_high, doc_term_matrix, dictionary, sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.113424  0.011516       1        1  23.801579
1     -0.093846 -0.044170       2        1   0.951963
2     -0.087500 -0.044031       3        1   0.297185
3      0.084350 -0.029467       4        1  10.074042
4     -0.040390  0.132675       5        1   4.423985
5      0.102320 -0.004316       6        1  12.716585
6      0.101671 -0.006362       7        1  14.391301
7     -0.119846 -0.041146       8        1   2.343534
8     -0.106304  0.136883       9        1   4.862674
9      0.046671  0.007412      10        1   4.670920
10    -0.081107 -0.045394      11        1   1.007195
11     0.082050  0.074073      12        1   8.598972
12     0.049090 -0.081225      13        1   4.879610
13    -0.072538 -0.048569      14        1   2.269772
14     0.021954 -0.017880      15        1   4.710692, topic_info=               Term       Freq      Total Category  logprob  loglift
477            duty  24.000000  24.000000  Default  30.0000  30.0000
102          mobile  51.000000  51.000000  Default  29.0000  29.0000
33          college  31.000000  31.000000  Default  28.0000  28.0000
115           order  55.000000  55.000000  Default  27.0000  27.0000
142  representative  40.000000  40.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
58        excellent   2.837774  49.219498  Topic15  -5.3489   0.2021
75      information   2.584911  41.383408  Topic15  -5.4422   0.2821
57            event   2.321491  29.997599  Topic15  -5.5497   0.4964
241     development   2.321439  21.963177  Topic15  -5.5497   0.8082
166          system   2.321415  27.700369  Topic15  -5.5497   0.5761

[1038 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2051      6  0.810563    aartex
454       1  0.340470   ability
454       4  0.180249   ability
454       6  0.100138   ability
454       7  0.140193   ability
...     ...       ...       ...
1875     12  0.926550     xerox
2085      6  0.810563      yarn
1120      9  0.688253      ymca
2413      1  0.852619      york
188       9  0.903411  yorkdale

[2298 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

### Only low performers

In [48]:
df_topic_model_low = df_topic_model.query("hp_class == 0.0")

In [49]:
corpus = df_topic_model_low["token_2"]
dictionary = corpora.Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [50]:
lda_low = models.LdaModel(corpus=doc_term_matrix, 
                           id2word=dictionary, 
                           num_topics=15, 
                           passes=30)

In [51]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_low, doc_term_matrix, dictionary, sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.076454  0.002711       1        1  13.051746
1      0.079210 -0.003959       2        1  17.335335
2     -0.059531  0.195463       3        1   3.011742
3      0.027031 -0.018411       4        1   4.093586
4      0.093486 -0.029960       5        1  10.537911
5     -0.129638  0.009661       6        1   1.044099
6      0.080049 -0.009374       7        1   9.498609
7      0.037628 -0.041272       8        1   6.630725
8     -0.192051 -0.006742       9        1   0.036287
9      0.063179  0.012321      10        1   6.281222
10     0.059992  0.028241      11        1   5.382369
11     0.023630 -0.020498      12        1   5.506738
12     0.096862 -0.000440      13        1  14.448504
13    -0.098794 -0.054310      14        1   2.233869
14    -0.157505 -0.063432      15        1   0.907263, topic_info=          Term        Freq       Total Category  logprob  loglift
191       food  122.000000  122.000000  Default  30.0000  30.0000
473     school  189.000000  189.000000  Default  29.0000  29.0000
22    computer   94.000000   94.000000  Default  28.0000  28.0000
0      ability  160.000000  160.000000  Default  27.0000  27.0000
112       time  150.000000  150.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
78    organize    1.548736  113.582832  Topic15  -5.4513   0.4074
614      roger    1.258629   21.497910  Topic15  -5.6587   1.8646
22    computer    1.183455   94.515808  Topic15  -5.7203   0.3222
410      learn    1.162732   77.048958  Topic15  -5.7379   0.5088
419  microsoft    1.054158   76.062790  Topic15  -5.8360   0.4237

[1137 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.081034  ability
0         2  0.162067  ability
0         3  0.024933  ability
0         4  0.081034  ability
0         5  0.099734  ability
...     ...       ...      ...
870       8  0.128815    youth
870      10  0.042938    youth
870      13  0.085877    youth
870      15  0.128815    youth
4606      4  0.834350     zara

[3328 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [52]:
# def get_most_prob_topic_w_prob(unseen_document, model = lda):
#     """
#     Given an unseen_document, and a trained LDA model, this function
#     finds the most likely topic (topic with the highest probability) from the 
#     topic distribution of the unseen document and returns the best topic with 
#     its probability. . 
    
#     Parameters
#     ------------
#     unseen_document : (str) 
#         the document to be labeled with a topic
#     model : (gensim ldamodel) 
#         the trained LDA model
    
#     Returns: 
#     -------------
#         (str) a string of the form 
#         `most likely topic label:probability of that label` 
    
#     Examples:
#     ----------
#     >> get_most_prob_topic("The research uses an HMM for discovering gene sequence.", 
#                             model = lda)
#     Science and Technology:0.435
#     """    
#     #preprocess 
#     preprocessed = preprocess(unseen_document)
    
#     #process into a doc_term_matrix
#     corpus = preprocessed
#     bow_vector = dictionary.doc2bow(corpus)
    
    
#     #find simliar articals
#     simliar_artical = model[bow_vector]
    
    
#     #change the topic name and sort tuple
#     items = []

#     for idx, prob in simliar_artical:
#         items.append((topic_labels[idx], prob))

#     items = sorted(items, key=lambda x: x[1], reverse=True)
#     out = str(items[0][0])+":"+str(items[0][1])
#     return out